In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 

# this gets the tables from the list of all Belgian beers, and does some cleaning 

In [35]:
url = "https://fr.wikipedia.org/wiki/Liste_de_bi%C3%A8res_belges"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

tables = soup.find_all("table", {"class": "wikitable"})

def remove_wikipedia_ref(text):
	return re.sub(r"\[\d+\]", "", text)

def remove_language_tags(text):
	return re.sub(r"\s*\((nl|en)\)", "", text)

def remove_trailing_commas(text):
	count_trailing = re.search(r",+$", text)
	if count_trailing:
		return text[:-len(count_trailing.group())]
	return text

def remove_pour_and_following_text(text):
	if "pour" in text:
		return text.split("pour")[0]
	return text

dataframes = []
for i, table in enumerate(tables):
    headers = [header.text.strip() for header in table.find_all("th")]
    rows = []
    for row in table.find_all("tr")[1:]:  # Skip header row
        cols = row.find_all(["td", "th"])
        cols = [col.text.strip() for col in cols]
        cols = [remove_wikipedia_ref(col) for col in cols]
        cols = [remove_trailing_commas(col) for col in cols]
        cols = [remove_language_tags(col) for col in cols]
        cols = [remove_pour_and_following_text(col) for col in cols]
        if cols:
            rows.append(cols)
    df = pd.DataFrame(rows, columns=headers)
    dataframes.append(df)

In [40]:
dataframes[1]['Brasserie'].unique()

array(['De Struise Brouwers', 'Stadsbrouwerij Aarschot',
       'Brasserie du Val de Sambre', 'Brasserie de Tubize',
       'Brasserie La Binchoise', 'Brouwerij Het Anker (',
       'Brasserie de Silly', 'Brasserie Lefebvre',
       'Brasserie Brootcoorens', 'Brasserie de Brunehaut',
       "Brasserie de l'Abbaye des Rocs", 'Brasserie Haacht',
       'Brasserie Riva', 'Brasserie Van Steenberge', "Abbaye d'Achel",
       'Brasserie La Binchoise ', 'Brasserie Cornelissen ',
       'Brasserie du Renard', 'Brasserie Roman',
       'Brasserie Affligem (Heineken)', 'Brasserie du Bocq',
       'Brasserie Nieuwhuys', 'Brasserie Louwaege', 'Brasserie Alvinne',
       'Brasserie Millevertus',
       'Brasserie Het Alternatief par la Brasserie Alvinne',
       'Brasserie Slaghmuylder', 'Brasserie Het Anker', 'Brasserie Deca',
       "Huisbrouwerij 't Pakhuis", 'De Dolle Brouwers',
       "Brasserie d'Écaussinnes", 'Brasserie de Bastogne',
       "Brasserie Grain d'Orge", 'Brasserie De Ryck',
    

# this gets the data from list of Belgian breweries, as well as some additional beer

In [72]:
url = "https://fr.wikipedia.org/wiki/Liste_de_brasseries_belges"

reponse = requests.get(url)
soup = BeautifulSoup(reponse.text, "html.parser")

tables = soup.find_all("table", {"class": "wikitable"})

dataframes_breweries = []
for i, table in enumerate(tables):
    headers = [header.text.strip() for header in table.find_all("th")]
    rows = []
    for row in table.find_all("tr")[1:]:
        cols = row.find_all(["td", "th"])
        cols = [col.text.strip() for col in cols]
        cols = [remove_wikipedia_ref(col) for col in cols]
        cols = [remove_trailing_commas(col) for col in cols]
        cols = [remove_language_tags(col) for col in cols]
        cols = cols[:6]
        if cols:
            rows.append(cols)
    df = pd.DataFrame(rows, columns=headers)
    dataframes_breweries.append(df.iloc[:, :4])

In [74]:
dataframes_breweries[0]

,Brasserie,Commune,Province,Principales bières
0,Brasserie de l'abbaye de Saint-Ghislain,Saint-Ghislain,Hainaut,Abbaye de Saint-Ghislain
1,Brasserie de l'abbaye de Saint-Sixte,Vleteren,Flandre-Occidentale,Westvleteren
2,Brasserie de l'abbaye des Rocs,Honnelles,Hainaut,Abbaye des Rocs
3,Brasserie de l'abbaye du Val-Dieu,Aubel,Liège,Val-Dieu
4,Brasserie de l'abbaye de Villers-la-Ville,Villers-la-Ville,Brabant wallon,Abbaye de Villers
5,Abbaye Notre-Dame d'Orval,Florenville,Luxembourg,Orval
6,Brasserie de l'abbaye Notre-Dame de Saint-Benoît,Hamont-Achel,Limbourg,Achel
7,Brasserie de l'abbaye Notre-Dame de Saint-Rémy,Rochefort,Namur,Rochefort
8,Brasserie de l'abbaye Notre-Dame de Scourmont,Chimay,Hainaut,Chimay
9,Brasserie de l'abbaye Notre-Dame du Sacré-Cœur...,Malle,Anvers,Westmalle


# we need to align the breweries from both datasets with the format of one of the two
for this we will use the transformers -- sentence_transformers
